# CX 183


In [1]:
#Import packages
from astropy.io import fits
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
from astropy.convolution import convolve, Box1DKernel
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
output_notebook()

Loading BokehJS ...

## Defining the extraction parameters

First we define the parameters to extract the desire spectra:
    - Name of the source
    - Center, low and upper limits of the aperture
    - background sample

# First Source

We begin with the first souce in the corner of the slit. 

In [7]:
sourcename = 'cx183sky'
center = 116
low = -2
high = 1
b_low = [center-center,center-center]
b_up = [70-center,100-center]
databasegeneral = 'database/apcx183'
filename = 'database/ap'+sourcename
fitsfilename = 'mos_science_sky_extracted_Q3.fits'

## Exposure time

First we multiply the 2D spectra times the exposure time

In [8]:
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    

iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)

##  Define apertures and backgrpund sample in the database

Now we need to change the database of the aperture to define the center. This is the only way I have found to do it automatically. After this define the parameters in the .par file

In [9]:
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagename= lines.split()[1]
        if 'center' in lines:
                numerocenter = lines.split()[2]
        if 'low' in lines:
                numerolow = lines.split()[2]
        if 'high' in lines:
                numerohigh = lines.split()[2]
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagename,sourcename)
filedata = filedata.replace(numerocenter,str(center))
filedata = filedata.replace(numerolow,str(low))
filedata = filedata.replace(numerohigh,str(high))
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

## Calling Apall

In [10]:
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')
#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
#try output
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

## Plotting with Bokeh

Plot the text file after doing dispcor and exporting the spectra to a text file. 

In [11]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
#Other way
#srfm = fits.open(sourcename+'.ms.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
#secondstar = srfm[0].data[0][0]
#xn = srfm[0].header["NAXIS1"]
#refx = srfm[0].header["CRVAL1"]
#step = srfm[0].header['CD1_1']
#cr = srfm[0].header['CRPIX1']
#
#xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#Create ColumnDataSource
#x = np.array(xlist)
#y = np.array(secondstar)

#Not sure if I need to do this
if os.path.exists(sourcename+'.dispcor.fits'):
    os.remove(sourcename+'.dispcor.fits')
iraf.onedspec()
iraf.dispcor(sourcename+'.ms.fits',sourcename+'.dispcor.fits')
iraf.wspectext(sourcename+'.dispcor.fits[*,1,1]',sourcename+'.txt',header='no')

x=[]
y=[]
with open(sourcename+'.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))


source = ColumnDataSource(data=dict(x=x,y=y))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra",plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
#Eraaseplot.line(xlist,secondstar)
plot.line('x','y',source=source)
show(plot)

cx183sky.ms.fits: Resampling using current coordinate system
cx183sky.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9998.7, dw =      2.6, nw = 2500


## Normalizing the spectrum

We use the iraf routine continuum. We can define a sample to correctly normalize the desire range of wavelenght


In [7]:
sample = '5000:9000'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.ms.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

## Plotting the normalize spectra

We first define the desired range for the initial zoom:

In [8]:
#Plotting
xr = (7700,8900)
yr = (0.5,1.4)

#Not sure if I need to do this
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))

x = np.array(x)
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)


cx59ssemcont.fits: Resampling using current coordinate system
cx59ssemcont.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9996.1, dw =      2.6, nw = 2499


GlyphRenderer(id='a2228d2d-3039-4bcf-8793-b42fedaa3f13', ...)

## Overplot lines

If desired we can define several know emission and absoprtion lines to overplot to the normalize spectra

In [9]:
##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name


diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,
            line('O I'):8446,line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('O I Triplet(72,74,75)'):7774,line('Na I(Interstellar lines 90 and 96)'):5895,line('He I'):7065,
            line('P16'):8502,line('P15'):8545,line('P14'):8598,line('P13'):8665,
            line('P12'):8750,line('P11'):8863,line('P10'):9015,line('P9'):9229,
            line('P8'):9546,line('P17'):8467,line('P18'):8438,line('P19'):8413,line('P20'):8392}
                    

for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)

In [10]:
show(plot)